In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

In [2]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

xgb_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', XGBRegressor())
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot_cols),
    ('regressor', SGDRegressor())
])

In [10]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithm=sgd_pipe)
    return thesis


In [11]:
thesis = full_run()

Finished experiment! Elapsed time: 0.07878851890563965, total Elapsed time: 0.07878851890563965, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.06782078742980957, total Elapsed time: 0.14660930633544922, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.05887794494628906, total Elapsed time: 0.20548725128173828, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.06783580780029297, total Elapsed time: 0.27332305908203125, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished experiment! Elapsed time: 0.06491518020629883, total Elapsed time: 0.3382382392883301, Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift tim

In [12]:
results = thesis.resultsToDF()

In [13]:
results[["SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE                  \
                                                       mean       std   min   
Drift type        Drift time     Drift magnitude                              
Incremental Drift Fully observed Large             3.096667  2.918805  1.65   
                                 Small             2.140000  0.371067  1.91   
                  Half observed  Large             9.269167  5.139311  3.06   
                                 Small             3.233333  1.212423  2.20   
                  Unobserved     Large             7.446667  4.064184  2.86   
                                 Small             3.067500  0.980669  2.20   
No Drift          No Drift       No Drift          2.055000  0.012910  2.04   
Sudden Drift      Fully observed Large             3.220000  3.175337  1.61   
                                 Small             2.139167  0.397228  1.92   
                  Half observed  Large             5.060000  5.005384  1.80   
                                 Small             2.535000  1.124565  1.98   
                  Unobserved     Large            10.423333  5.773101  3.22   
                                 Small             3.326667  1.417856  2.19   

                                                         
                                                    max  
Drift type        Drift time     Drift magnitude         
Incremental Drift Fully observed Large            10.77  
                                 Small             3.24  
                  Half observed  Large            21.74  
                                 Small             6.58  
                  Unobserved     Large            17.53  
                                 Small             5.57  
No Drift          No Drift       No Drift          2.07  
Sudden Drift      Fully observed Large            11.52  
                                 Small             3.31  
                  Half observed  Large            18.40  
                                 Small             5.70  
                  Unobserved     Large            24.95  
                                 Small             7.29

In [7]:
results.sort_values("SMAPE", ascending=False)   

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
142,6.0,Incremental Drift,Large,Medium,Half observed,162413.496101,[28.51]
68,0.0,Incremental Drift,Large,Medium,Half observed,69922.463470,[21.28]
143,6.0,Incremental Drift,Large,Medium,Unobserved,120920.732804,[19.91]
31,NaN,Incremental Drift,Large,Medium,Half observed,50532.583939,[19.45]
69,0.0,Incremental Drift,Large,Medium,Unobserved,90685.703338,[18.55]
...,...,...,...,...,...,...,...
27,NaN,Incremental Drift,Large,Important,Fully observed,14.963985,[2.86]
83,3.0,Sudden Drift,Large,Important,Fully observed,14.710612,[2.84]
9,NaN,Sudden Drift,Large,Important,Fully observed,14.815449,[2.84]
138,6.0,Incremental Drift,Large,Important,Fully observed,14.940066,[2.84]
